from functions2 import Baum_Welch, forward_HMM, backward_HMM, compute_all_conditional
import numpy as np

A = np.array([[0.6, 0.4], [0.3, 0.7]])
B = np.array([[0.5, 0.5], [0.1, 0.9]])
pi = np.array([0.2, 0.8])
observed = np.array([1, 0, 1])
B_start = np.zeros((2, 2)) + 0.5

alpha, c = forward_HMM(A, B, pi, observed)
beta_hat, beta = backward_HMM(A, B, observed, c)
gamma = compute_all_conditional(alpha, beta)


# text experiment


In [25]:
import random
from src.CipherUtils import CipherGenerator
from src.CipherUtils import TextDecoder, TextEncoder


hidden_sequence = "people of western europe a landing was made this morning on the coast of france by troops kangaroo jokes quasi vile xilophone zenit "
hidden_sequence = " in germany it seems to be pretty much automatic pretty much all the time in france and spain it all depends presumably on social subtleties that you have to be french or spanish to understand in italy why would you even bother when and how much to tip is a question that has been vexing visitors to europe for as long as people have been travelling around the continent outside their own country it seems even europeans don t know the answer according to new polling by yougov in six eu countries britain and the us where as most visitors know but may be reluctant to acknowledge gratuities may make up more than half your waitperson s income europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typically tipped almost the same as the us in the uk where an optional service charge of about is usually included said they left a gratuity the figure in spain where service is often included in restaurant bills but diners can leave optional tips was while in france where every price on a restaurant menu already includes for service of people said they generally tipped on top even in sweden where tips are generally not expected the figure was but only of italians said they would typically leave a gratuity after a meal out with a rather greater proportion admitting they never left a cent a startling of respondents in the us however and of germans by far the most in europe confessed they would tip sometimes or often even if the service was terrible indicating that for some tipping is not about quality of service at all the findings of the survey will come as a surprise in germany a country that does not generally think of itself as a nation of happy distributors of the word in a similar vein to pourboire in french means drinking money"
hidden_sequence = " hello banana xilophone key queue zebra cock pussy tits dandy fart though jolly world mum "
hidden_sequence = " hello"


cipher_generator = CipherGenerator()
cipher = cipher_generator.generate_cipher()
encoder = TextEncoder()
observed_sequence = encoder.encode_text(hidden_sequence, cipher=cipher)


print(observed_sequence)

 dneex


In [26]:
from prove.src.Probability import ProbabilityMatrix

with open("texts/moby_dick.txt", "r") as file:
    text = file.read()

alphabet = list("abcdefghijklmnopqrstuvwxyz ")


pm = ProbabilityMatrix(text=text, alphabet=alphabet)
unknown_chars = pm.unknown_chars()
pm.preprocess_text(unknown_chars=unknown_chars)

# compute probabilities
pm.compute_matrix_spaces()

# classe usata per MCMC qui in teori non serve piu

from src.ProbabilityMatrix import ProbabilityMatrix
from src.CipherUtils import TextPreProcessor

preprocess = TextPreProcessor()
text = preprocess.lower(text)
text = preprocess.remove_additional_spaces(text)
text = preprocess.remove_unknown_chars(
text=text, unknown_chars=preprocess.unknown_chars(text)
)

probability = ProbabilityMatrix(text=text)
probability.compute_probability_table()


In [27]:
def map_alphabet_to_numbers():
    alphabet = "abcdefghijklmnopqrstuvwxyz "
    mapping = {char: i for i, char in enumerate(alphabet)}
    return mapping

In [28]:
def translate_string_to_numbers(text, mapping):
    numbers = [mapping[char] for char in text]
    return numbers

In [29]:
hidden_ = translate_string_to_numbers(
    hidden_sequence, mapping=map_alphabet_to_numbers()
)
observed_ = translate_string_to_numbers(
    observed_sequence, mapping=map_alphabet_to_numbers()
)

In [30]:
transition_probs = pm.matrix

In [31]:
from functions2 import forward_HMM
import numpy as np

a, ah, c = forward_HMM(
    A=transition_probs,
    B=np.zeros((27, 27)) + 1 / 27,
    pi=transition_probs[-1, :],
    observed=observed_,
)

In [32]:
from functions2 import backward_HMM

b, bh = backward_HMM(
    A=pm.matrix, B=np.zeros((27, 27)) + 1 / 27, observed=observed_, c=c
)

In [33]:
from functions2 import Baum_Welch
import numpy as np

emission = Baum_Welch(
    A=pm.matrix,
    B_start=np.zeros((27, 27)) + 1 / 27,
    pi=pm.matrix[-1, :],
    observed=observed_,
    maxIter=1000,
)

In [34]:
emission

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 1.00000000e+0

In [35]:
emission.argmax(axis=1)

array([ 4, 23, 23, 23,  4, 23, 23, 13,  4, 23, 23, 23, 23,  4, 23, 23, 23,
        4, 23,  3, 23, 23,  3, 23, 23, 23, 26])